<a href="https://colab.research.google.com/github/SinnottKayleigh/B2B-Sales-Algos/blob/main/MIFID_Classifications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This snippet of code can be implemented in a larger algorithm, to clearly identify specific MIFID classifications for clients trading options and forwrads.

Elective Professional:
- Must meet 2/3 of the following criteria:
- The client has carried out transactions, in a significant size, on the relevant market at an ***average frequency of 10 per quarter, over the last 4 quarters. ***
- The size of the clients financial instrument portfolio defined as ***including cash deposits and financial instruments exceeds EUR 500,000. ***
- The client works or has worked in the financial sector for ***at least 1 year in a professional position***, which requires knowledge of transactions or services envisaged.

Per Se Professional:
- Meet ***2/3*** of the following:
- Balance sheet total of EUR ***20,000,000***
- Net turnover of EUR ***40,000,000***
- Own funds of EUR ***2,000,000***



Or an entity to operate in the financial markets:
- A credit instiutional
- An investment firm
- Any other authorised or regulated financial institution
- A collective investment scheme or the management company of such a scheme
- A pension fund or the management company of a pension fund
- A commodity or commodity derivatives dealer
- A local authority
- Any other institutional Investor

FUNCTION ClassifyMiFIDClient(clientData):
    // Initialize classification flags
    IS_ELECTIVE_PROFESSIONAL = false
    IS_PER_SE_PROFESSIONAL = false
    IS_INSTITUTIONAL = false

    // First check if client is an institutional entity
    IF CheckInstitutionalStatus(clientData):
        IS_INSTITUTIONAL = true
        RETURN {
            "classification": "Institutional Professional",
            "reason": "Qualified as institutional entity",
            "category": clientData.entityType
        }

    // Check Per Se Professional criteria
    perSeCriteriaMet = 0
    IF clientData.balanceSheet >= 20000000:
        perSeCriteriaMet += 1
    IF clientData.netTurnover >= 40000000:
        perSeCriteriaMet += 1
    IF clientData.ownFunds >= 2000000:
        perSeCriteriaMet += 1

    IF perSeCriteriaMet >= 2:
        IS_PER_SE_PROFESSIONAL = true
        RETURN {
            "classification": "Per Se Professional",
            "criteriamet": perSeCriteriaMet,
            "metrics": {
                "balanceSheet": clientData.balanceSheet,
                "netTurnover": clientData.netTurnover,
                "ownFunds": clientData.ownFunds
            }
        }

    // Check Elective Professional criteria
    electiveCriteriaMet = 0

    // Check transaction frequency
    IF CheckTransactionFrequency(clientData.transactions):
        electiveCriteriaMet += 1

    // Check portfolio size
    IF CheckPortfolioSize(clientData.portfolio):
        electiveCriteriaMet += 1

    // Check professional experience
    IF CheckProfessionalExperience(clientData.experience):
        electiveCriteriaMet += 1

    IF electiveCriteriaMet >= 2:
        IS_ELECTIVE_PROFESSIONAL = true
        RETURN {
            "classification": "Elective Professional",
            "criteriamet": electiveCriteriaMet,
            "metrics": {
                "transactionFrequency": GetTransactionMetrics(clientData.transactions),
                "portfolioSize": clientData.portfolio.totalValue,
                "professionalExperience": clientData.experience.duration
            }
        }

    // If no professional criteria met
    RETURN {
        "classification": "Retail",
        "reason": "Did not meet professional criteria",
        "electiveCriteriaMet": electiveCriteriaMet,
        "perSeCriteriaMet": perSeCriteriaMet
    }

FUNCTION CheckTransactionFrequency(transactions):
    quarterlyTransactions = []
    FOR EACH quarter IN last4Quarters:
        significantTransactions = FilterSignificantTransactions(transactions[quarter])
        quarterlyTransactions.append(COUNT(significantTransactions))
    
    averageTransactions = AVERAGE(quarterlyTransactions)
    RETURN averageTransactions >= 10

FUNCTION CheckPortfolioSize(portfolio):
    totalValue = portfolio.cashDeposits + portfolio.financialInstruments
    RETURN totalValue >= 500000

FUNCTION CheckProfessionalExperience(experience):
    IF experience.sector == "financial" AND
       experience.duration >= 1 AND
       experience.position == "professional":
        RETURN true
    RETURN false

FUNCTION CheckInstitutionalStatus(clientData):
    institutionalTypes = [
        "credit_institution",
        "investment_firm",
        "regulated_financial_institution",
        "collective_investment_scheme",
        "pension_fund",
        "commodity_dealer",
        "local_authority",
        "institutional_investor"
    ]
    
    RETURN clientData.entityType IN institutionalTypes

FUNCTION FilterSignificantTransactions(transactions):
    // Define significance threshold based on market standards
    significantThreshold = DefineSignificanceThreshold(transactions.market)
    RETURN FILTER transactions WHERE value >= significantThreshold

In [6]:
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import List, Dict, Optional
import pandas as pd
from enum import Enum
import numpy as np

class ClientClassification(Enum):
    RETAIL = "Retail"
    ELECTIVE_PROFESSIONAL = "Elective Professional"
    PER_SE_PROFESSIONAL = "Per Se Professional"
    INSTITUTIONAL = "Institutional Professional"

class EntityType(Enum):
    CREDIT_INSTITUTION = "credit_institution"
    INVESTMENT_FIRM = "investment_firm"
    FINANCIAL_INSTITUTION = "regulated_financial_institution"
    INVESTMENT_SCHEME = "collective_investment_scheme"
    PENSION_FUND = "pension_fund"
    COMMODITY_DEALER = "commodity_dealer"
    LOCAL_AUTHORITY = "local_authority"
    INSTITUTIONAL_INVESTOR = "institutional_investor"
    OTHER = "other"

@dataclass
class Transaction:
    date: datetime
    value: float
    market: str
    type: str

@dataclass
class Portfolio:
    cash_deposits: float
    financial_instruments: float

@dataclass
class ProfessionalExperience:
    sector: str
    duration: float  # in years
    position: str

@dataclass
class ClientData:
    client_id: str
    entity_type: EntityType
    transactions: List[Transaction]
    portfolio: Portfolio
    experience: Optional[ProfessionalExperience]
    balance_sheet: Optional[float]
    net_turnover: Optional[float]
    own_funds: Optional[float]

class MiFIDClassifier:
    def __init__(self):
        self.SIGNIFICANT_TRANSACTION_THRESHOLDS = {
            "fx_spot": 100000,
            "fx_forward": 250000,
            "fx_option": 500000,
            "default": 100000
        }

    def classify_client(self, client_data: ClientData) -> Dict:
        """Main classification method"""

        if self._check_institutional_status(client_data.entity_type):
            return {
                "classification": ClientClassification.INSTITUTIONAL.value,
                "reason": "Qualified as institutional entity",
                "category": client_data.entity_type.value
            }

        per_se_result = self._check_per_se_professional(client_data)
        if per_se_result["qualified"]:
            return {
                "classification": ClientClassification.PER_SE_PROFESSIONAL.value,
                "criteria_met": per_se_result["criteria_met"],
                "metrics": per_se_result["metrics"]
            }

        elective_result = self._check_elective_professional(client_data)
        if elective_result["qualified"]:
            return {
                "classification": ClientClassification.ELECTIVE_PROFESSIONAL.value,
                "criteria_met": elective_result["criteria_met"],
                "metrics": elective_result["metrics"]
            }

        return {
            "classification": ClientClassification.RETAIL.value,
            "reason": "Did not meet professional criteria",
            "per_se_criteria_met": per_se_result["criteria_met"],
            "elective_criteria_met": elective_result["criteria_met"]
        }

    def _check_institutional_status(self, entity_type: EntityType) -> bool:
        """Check if client is an institutional entity"""
        return entity_type != EntityType.OTHER

    def _check_per_se_professional(self, client_data: ClientData) -> Dict:
        """Check Per Se Professional criteria"""
        criteria_met = 0
        metrics = {
            "balance_sheet": client_data.balance_sheet,
            "net_turnover": client_data.net_turnover,
            "own_funds": client_data.own_funds
        }

        if client_data.balance_sheet and client_data.balance_sheet >= 20_000_000:
            criteria_met += 1
        if client_data.net_turnover and client_data.net_turnover >= 40_000_000:
            criteria_met += 1
        if client_data.own_funds and client_data.own_funds >= 2_000_000:
            criteria_met += 1

        return {
            "qualified": criteria_met >= 2,
            "criteria_met": criteria_met,
            "metrics": metrics
        }

    def _check_elective_professional(self, client_data: ClientData) -> Dict:
        """Check Elective Professional criteria"""
        criteria_met = 0
        metrics = {}

        transaction_result = self._check_transaction_frequency(client_data.transactions)
        if transaction_result["qualified"]:
            criteria_met += 1
        metrics["transactions"] = transaction_result["metrics"]

        portfolio_result = self._check_portfolio_size(client_data.portfolio)
        if portfolio_result["qualified"]:
            criteria_met += 1
        metrics["portfolio"] = portfolio_result["metrics"]

        if client_data.experience:
            experience_result = self._check_professional_experience(client_data.experience)
            if experience_result["qualified"]:
                criteria_met += 1
            metrics["experience"] = experience_result["metrics"]

        return {
            "qualified": criteria_met >= 2,
            "criteria_met": criteria_met,
            "metrics": metrics
        }

    def _check_transaction_frequency(self, transactions: List[Transaction]) -> Dict:
        """Check if transaction frequency meets criteria"""
        if not transactions:
            return {"qualified": False, "metrics": {"avg_quarterly_transactions": 0}}

        df = pd.DataFrame([
            {
                "date": t.date,
                "value": t.value,
                "market": t.market
            } for t in transactions
        ])

        df["is_significant"] = df.apply(
            lambda x: x["value"] >= self.SIGNIFICANT_TRANSACTION_THRESHOLDS.get(
                x["market"], self.SIGNIFICANT_TRANSACTION_THRESHOLDS["default"]
            ),
            axis=1
        )

        df["quarter"] = df["date"].dt.to_period("Q")
        quarterly_counts = df[df["is_significant"]].groupby("quarter").size()

        last_4_quarters = quarterly_counts.tail(4)
        avg_quarterly_transactions = last_4_quarters.mean() if len(last_4_quarters) > 0 else 0

        return {
            "qualified": avg_quarterly_transactions >= 10,
            "metrics": {
                "avg_quarterly_transactions": avg_quarterly_transactions,
                "quarterly_breakdown": quarterly_counts.to_dict()
            }
        }

    def _check_portfolio_size(self, portfolio: Portfolio) -> Dict:
        """Check if portfolio size meets criteria"""
        total_value = portfolio.cash_deposits + portfolio.financial_instruments
        return {
            "qualified": total_value >= 500_000,
            "metrics": {
                "total_value": total_value,
                "cash_deposits": portfolio.cash_deposits,
                "financial_instruments": portfolio.financial_instruments
            }
        }

    def _check_professional_experience(self, experience: ProfessionalExperience) -> Dict:
        """Check if professional experience meets criteria"""
        qualified = (
            experience.sector.lower() == "financial" and
            experience.duration >= 1 and
            experience.position.lower() == "professional"
        )
        return {
            "qualified": qualified,
            "metrics": {
                "sector": experience.sector,
                "duration": experience.duration,
                "position": experience.position
            }
        }

def example_usage():
    client_data = ClientData(
        client_id="12345",
        entity_type=EntityType.OTHER,
        transactions=[
            Transaction(
                date=datetime.now() - timedelta(days=x),
                value=150000,
                market="fx_spot",
                type="spot"
            ) for x in range(0, 365, 7)
        ],
        portfolio=Portfolio(
            cash_deposits=300000,
            financial_instruments=300000
        ),
        experience=ProfessionalExperience(
            sector="financial",
            duration=1.5,
            position="professional"
        ),
        balance_sheet=25000000,
        net_turnover=45000000,
        own_funds=3000000
    )

    classifier = MiFIDClassifier()
    result = classifier.classify_client(client_data)

    print("Classification Result:")
    print(result)

if __name__ == "__main__":
    example_usage()

Classification Result:
{'classification': 'Per Se Professional', 'criteria_met': 3, 'metrics': {'balance_sheet': 25000000, 'net_turnover': 45000000, 'own_funds': 3000000}}


In [9]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import calendar

class MiFIDVisualizer:
    def __init__(self, classifier_results: Dict, client_data: ClientData):
        self.results = classifier_results
        self.client_data = client_data
        self.set_style()

    def set_style(self):
        """Set the style for matplotlib visualizations"""
        plt.style.use('default')  # Use default style instead of seaborn

    def create_dashboard(self):
        """Create a comprehensive dashboard of visualizations"""
        print(f"\nMiFID Classification Dashboard for Client {self.client_data.client_id}")
        print("=" * 50)

        self.plot_transaction_analysis()
        self.plot_portfolio_breakdown()
        self.plot_criteria_summary()
        self.plot_transaction_heatmap()
        if 'metrics' in self.results:
            self.plot_professional_criteria_radar()

    def plot_transaction_analysis(self):
        """Visualize transaction patterns"""
        df = pd.DataFrame([
            {
                'date': t.date,
                'value': t.value,
                'market': t.market,
                'type': t.type
            } for t in self.client_data.transactions
        ])

        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Transaction Volume Over Time',
                'Transaction Distribution by Market',
                'Monthly Transaction Count',
                'Transaction Size Distribution'
            )
        )

        fig.add_trace(
            go.Scatter(
                x=df['date'],
                y=df['value'],
                mode='lines+markers',
                name='Transaction Value'
            ),
            row=1, col=1
        )

        market_dist = df.groupby('market')['value'].sum()
        fig.add_trace(
            go.Pie(
                labels=market_dist.index,
                values=market_dist.values,
                name='Market Distribution'
            ),
            row=1, col=2
        )

        monthly_count = df.groupby(df['date'].dt.strftime('%B'))['value'].count()
        fig.add_trace(
            go.Bar(
                x=monthly_count.index,
                y=monthly_count.values,
                name='Monthly Count'
            ),
            row=2, col=1
        )

        fig.add_trace(
            go.Histogram(
                x=df['value'],
                name='Size Distribution'
            ),
            row=2, col=2
        )

        fig.update_layout(
            height=800,
            showlegend=False,
            title_text="Transaction Analysis Dashboard",
            title_x=0.5
        )
        fig.show()

    def plot_portfolio_breakdown(self):
        """Visualize portfolio composition"""
        portfolio = self.client_data.portfolio

        fig = go.Figure()

        fig.add_trace(go.Pie(
            labels=['Cash Deposits', 'Financial Instruments'],
            values=[portfolio.cash_deposits, portfolio.financial_instruments],
            hole=0.4
        ))

        fig.update_layout(
            title={
                'text': 'Portfolio Composition',
                'x': 0.5
            },
            annotations=[{
                'text': f'Total: €{portfolio.cash_deposits + portfolio.financial_instruments:,.0f}',
                'showarrow': False,
                'font': {'size': 20}
            }]
        )

        fig.show()

    def plot_criteria_summary(self):
        """Visualize criteria fulfillment"""
        if 'metrics' in self.results:
            metrics = self.results['metrics']

            criteria_data = {
                'Criteria': [
                    'Transaction Frequency',
                    'Portfolio Size',
                    'Professional Experience'
                ],
                'Required': [10, 500000, 1],
                'Actual': [
                    metrics.get('transactions', {}).get('avg_quarterly_transactions', 0),
                    metrics.get('portfolio', {}).get('total_value', 0),
                    metrics.get('experience', {}).get('duration', 0)
                ]
            }

            df = pd.DataFrame(criteria_data)

            fig = go.Figure()

            fig.add_trace(go.Bar(
                name='Required',
                x=df['Criteria'],
                y=df['Required'],
                marker_color='lightgray'
            ))

            fig.add_trace(go.Bar(
                name='Actual',
                x=df['Criteria'],
                y=df['Actual'],
                marker_color='rgb(66, 135, 245)'
            ))

            fig.update_layout(
                title={
                    'text': 'Criteria Fulfillment Summary',
                    'x': 0.5
                },
                barmode='group'
            )

            fig.show()

    def plot_transaction_heatmap(self):
        """Create a heatmap of transaction activity"""
        df = pd.DataFrame([
            {
                'date': t.date,
                'value': t.value
            } for t in self.client_data.transactions
        ])

        df['month'] = df['date'].dt.month
        df['day'] = df['date'].dt.day

        heatmap_data = df.pivot_table(
            values='value',
            index='day',
            columns='month',
            aggfunc='sum'
        ).fillna(0)

        fig = go.Figure(data=go.Heatmap(
            z=heatmap_data.values,
            x=[calendar.month_abbr[m] for m in heatmap_data.columns],
            y=heatmap_data.index,
            colorscale='Viridis'
        ))

        fig.update_layout(
            title={
                'text': 'Transaction Activity Heatmap',
                'x': 0.5
            },
            xaxis_title='Month',
            yaxis_title='Day of Month'
        )

        fig.show()

    def plot_professional_criteria_radar(self):
        """Create a radar chart for professional criteria"""
        if self.client_data.balance_sheet and self.client_data.net_turnover and self.client_data.own_funds:
            criteria_values = [
                (self.client_data.balance_sheet / 20000000) * 100,
                (self.client_data.net_turnover / 40000000) * 100,
                (self.client_data.own_funds / 2000000) * 100
            ]

            fig = go.Figure()

            fig.add_trace(go.Scatterpolar(
                r=criteria_values,
                theta=['Balance Sheet', 'Net Turnover', 'Own Funds'],
                fill='toself',
                name='Actual vs Required (%)'
            ))

            fig.update_layout(
                polar=dict(
                    radialaxis=dict(
                        visible=True,
                        range=[0, max(criteria_values) + 10]
                    )),
                showlegend=False,
                title={
                    'text': 'Professional Criteria Radar Chart',
                    'x': 0.5
                }
            )

            fig.show()